# Mini Git - DSA Concepts and Complexity Analysis

This notebook provides detailed analysis of all Data Structures and Algorithms used in Mini Git.

## Table of Contents
1. Hash Tables
2. Trees (N-ary)
3. Linked Lists
4. Graphs (DAG)
5. Dynamic Programming (LCS)
6. Graph Traversal
7. Complexity Summary

## 1. Hash Tables

### Application in Mini Git:
- **Object Storage**: Content-addressable storage using SHA-1 hashing
- **Staging Area**: File path → content hash mapping
- **Branch Management**: Branch name → commit hash mapping

### Hash Function: SHA-1
```
Input: File content (bytes)
Output: 40-character hexadecimal string
Property: Same content → same hash (deterministic)
```

### Time Complexity:
- **Insert**: O(1) average case
- **Lookup**: O(1) average case
- **Delete**: O(1) average case
- **Hash computation**: O(n) where n is content size

### Space Complexity:
- O(k) where k is number of stored items

In [1]:
# Hash Table Demo
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

from src.hash_object import hash_file
import time

# Demonstrate hash function
content1 = b"Hello, Mini Git!"
content2 = b"Hello, Mini Git!"  # Same content
content3 = b"Hello, Mini Git!!"  # Different content

from src.hash_object import HashObject
hash1 = HashObject.hash_content(content1)
hash2 = HashObject.hash_content(content2)
hash3 = HashObject.hash_content(content3)

print("SHA-1 Hash Demonstration:")
print(f"Content 1: {content1}")
print(f"Hash 1:    {hash1}\n")
print(f"Content 2: {content2}")
print(f"Hash 2:    {hash2}")
print(f"Same? {hash1 == hash2}\n")
print(f"Content 3: {content3}")
print(f"Hash 3:    {hash3}")
print(f"Different? {hash1 != hash3}")

SHA-1 Hash Demonstration:
Content 1: b'Hello, Mini Git!'
Hash 1:    0680c08416694f75391a54b523f7f9784c2a089f

Content 2: b'Hello, Mini Git!'
Hash 2:    0680c08416694f75391a54b523f7f9784c2a089f
Same? True

Content 3: b'Hello, Mini Git!!'
Hash 3:    f35db4e0a8dd74c6932f76c011780e5254ceec11
Different? True


## 2. N-ary Trees

### Application:
- **Directory Structure**: Represents file hierarchy
- Each node can have multiple children (files/subdirectories)

### Tree Structure:
```
root (.)
├── file1.txt
├── file2.txt
└── subdir/
    ├── file3.txt
    └── file4.txt
```

### Traversal: Depth-First Search (DFS)
- Used for serialization/deserialization
- Used for listing all files

### Time Complexity:
- **Add file**: O(d) where d is path depth
- **Find file**: O(d)
- **List all files**: O(n) where n is total nodes (DFS)
- **Serialize**: O(n)

### Space Complexity:
- O(n) for storing tree
- O(h) for recursion stack where h is tree height

In [2]:
# Tree Demo
from src.tree import DirectoryTree

# Build tree
tree = DirectoryTree()
tree.add_file("file1.txt", "hash1")
tree.add_file("dir1/file2.txt", "hash2")
tree.add_file("dir1/subdir/file3.txt", "hash3")
tree.add_file("dir2/file4.txt", "hash4")

print("Directory Tree:")
files = tree.list_all_files()
for f in files:
    print(f"  {f}")

print(f"\nTotal files: {len(files)}")
print("Tree traversal: DFS (Depth-First Search)")

Directory Tree:
  file1.txt
  dir1/file2.txt
  dir1/subdir/file3.txt
  dir2/file4.txt

Total files: 4
Tree traversal: DFS (Depth-First Search)


## 3. Singly Linked List

### Application:
- **Commit History**: Each commit points to its parent
- Forms a chain from newest to oldest commit

### Structure:
```
HEAD → [Commit 3] → [Commit 2] → [Commit 1] → NULL
       (latest)                    (initial)
```

### Node Structure:
```python
class Commit:
    data: tree_hash, message, author, timestamp
    next: parent_hash (pointer to parent commit)
```

### Operations:
- **Insert at head**: O(1) - new commit always becomes head
- **Traverse**: O(n) - follow parent pointers
- **Find commit**: O(n) in worst case

### Time Complexity:
- **Get history**: O(k) where k is number of commits to retrieve
- **Count commits**: O(n) where n is total commits

### Space Complexity:
- O(n) for storing all commits

In [3]:
# Linked List Demo
from src.commit import Commit

# Simulate linked list
commit1 = Commit("tree1", None, "Initial commit", "User")
commit1.hash = "abc123"

commit2 = Commit("tree2", "abc123", "Second commit", "User")
commit2.hash = "def456"

commit3 = Commit("tree3", "def456", "Third commit", "User")
commit3.hash = "ghi789"

print("Linked List Structure:")
print(f"HEAD: {commit3.hash} → parent: {commit3.parent_hash}")
print(f"      {commit2.hash} → parent: {commit2.parent_hash}")
print(f"      {commit1.hash} → parent: {commit1.parent_hash or 'NULL'}")

# Traverse linked list
print("\nTraversal (newest to oldest):")
commits_dict = {
    "ghi789": commit3,
    "def456": commit2,
    "abc123": commit1
}

current = "ghi789"
count = 0
while current:
    commit = commits_dict[current]
    print(f"  {count+1}. {commit.message}")
    current = commit.parent_hash
    count += 1

print(f"\nTime complexity: O({count})")

Linked List Structure:
HEAD: ghi789 → parent: def456
      def456 → parent: abc123
      abc123 → parent: NULL

Traversal (newest to oldest):
  1. Third commit
  2. Second commit
  3. Initial commit

Time complexity: O(3)


## 4. Directed Acyclic Graph (DAG)

### Application:
- **Commit Graph**: Commits and their relationships
- **Branch Structure**: Branches pointing to different commits

### Properties:
- **Directed**: Parent → child relationship
- **Acyclic**: No cycles (can't go back in time)
- **Multiple paths**: Different branches can diverge

### Graph Structure:
```
        main      feature
         ↓          ↓
      [C3]       [C4]
         \        /
          \      /
           [C2]
              |
            [C1] ← base
```

### Algorithms:
1. **Common Ancestor Finding**:
   - Time: O(n + m) where n, m are branch depths
   - Space: O(n) for storing ancestors
   
2. **Branch Operations**:
   - Create: O(1)
   - Switch: O(1)
   - List: O(b) where b is number of branches

In [4]:
# DAG Demo - Common Ancestor Algorithm

def find_common_ancestor_demo():
    # Simulate commit graph
    commits = {
        "C1": None,      # root
        "C2": "C1",      # C1 → C2
        "C3": "C2",      # C2 → C3 (main branch)
        "C4": "C2"       # C2 → C4 (feature branch)
    }
    
    def get_ancestors(commit):
        """Get all ancestors of a commit."""
        ancestors = set()
        current = commit
        while current:
            ancestors.add(current)
            current = commits[current]
        return ancestors
    
    def find_common(commit1, commit2):
        """Find common ancestor."""
        ancestors1 = get_ancestors(commit1)
        current = commit2
        while current:
            if current in ancestors1:
                return current
            current = commits[current]
        return None
    
    # Find common ancestor of C3 (main) and C4 (feature)
    common = find_common("C3", "C4")
    
    print("Commit DAG:")
    print("  C3 (main)    C4 (feature)")
    print("     \\          /")
    print("      \\        /")
    print("         C2")
    print("          |")
    print("         C1")
    print(f"\nCommon ancestor of C3 and C4: {common}")
    print("\nAlgorithm:")
    print("1. Get all ancestors of C3: {C3, C2, C1}")
    print("2. Traverse C4's ancestors until found in set")
    print("3. Found: C2")
    print("\nTime: O(n + m), Space: O(n)")

find_common_ancestor_demo()

Commit DAG:
  C3 (main)    C4 (feature)
     \          /
      \        /
         C2
          |
         C1

Common ancestor of C3 and C4: C2

Algorithm:
1. Get all ancestors of C3: {C3, C2, C1}
2. Traverse C4's ancestors until found in set
3. Found: C2

Time: O(n + m), Space: O(n)


## 5. Dynamic Programming - Longest Common Subsequence (LCS)

### Application:
- **Diff Algorithm**: Finding line-by-line differences between files

### Algorithm:
Given two sequences X and Y, find the longest subsequence common to both.

### DP Table:
```
dp[i][j] = LCS length of X[0..i] and Y[0..j]

Recurrence:
if X[i] == Y[j]:
    dp[i][j] = dp[i-1][j-1] + 1
else:
    dp[i][j] = max(dp[i-1][j], dp[i][j-1])
```

### Time Complexity: O(m × n)
- m = number of lines in file 1
- n = number of lines in file 2

### Space Complexity: O(m × n)
- For DP table

### Backtracking: O(m + n)
- To reconstruct diff from DP table

In [5]:
# LCS Demo
from src.diff import compute_lcs_length, compute_diff

text1 = """Line 1
Line 2
Line 3"""

text2 = """Line 1
Modified Line 2
Line 3
Line 4"""

lines1 = text1.splitlines()
lines2 = text2.splitlines()

# Compute LCS
dp = compute_lcs_length(lines1, lines2)

print("DP Table for LCS:")
print("    "+ " ".join(f"{i:3}" for i in range(len(lines2)+1)))
for i in range(len(lines1)+1):
    print(f"{i:2} ", end="")
    for j in range(len(lines2)+1):
        print(f"{dp[i][j]:3}", end=" ")
    print()

print(f"\nLCS Length: {dp[len(lines1)][len(lines2)]}")

# Compute diff
diff = compute_diff(text1, text2)
print("\nDiff Result:")
for line in diff:
    print(line)

DP Table for LCS:
      0   1   2   3   4
 0   0   0   0   0   0 
 1   0   1   1   1   1 
 2   0   1   1   1   1 
 3   0   1   1   2   2 

LCS Length: 2

Diff Result:
  Line 1
- Line 2
+ Modified Line 2
  Line 3
+ Line 4


## 6. Graph Traversal Algorithms

### Depth-First Search (DFS):
- **Tree serialization**: Recursive traversal
- **Commit history**: Following parent pointers

### Breadth-First Search (BFS):
- Could be used for level-order branch exploration
- Not currently implemented but possible extension

### Path Finding:
- **Common ancestor**: Set intersection approach
- **History traversal**: Following parent links

In [6]:
# DFS Demo on Tree
from src.tree import TreeNode

# Build a sample tree
root = TreeNode(".", False)
file1 = TreeNode("file1.txt", True, "hash1")
dir1 = TreeNode("dir1", False)
file2 = TreeNode("file2.txt", True, "hash2")

root.add_child(file1)
root.add_child(dir1)
dir1.add_child(file2)

def dfs_print(node, depth=0):
    """DFS traversal with visualization."""
    indent = "  " * depth
    node_type = "FILE" if node.is_file else "DIR"
    print(f"{indent}{node.name} ({node_type})")
    
    if not node.is_file:
        for child in node.children.values():
            dfs_print(child, depth + 1)

print("DFS Traversal:")
dfs_print(root)
print("\nTime: O(n), Space: O(h) where h = height")

DFS Traversal:
. (DIR)
  file1.txt (FILE)
  dir1 (DIR)
    file2.txt (FILE)

Time: O(n), Space: O(h) where h = height


## 7. Complexity Summary Table

| Operation | Data Structure | Time | Space |
|-----------|---------------|------|-------|
| Hash file | Hash function | O(n) | O(1) |
| Store object | Hash table | O(1) | O(1) |
| Retrieve object | Hash table | O(1) | O(1) |
| Add file to tree | N-ary tree | O(d) | O(d) |
| Find file in tree | N-ary tree | O(d) | O(1) |
| List all files | N-ary tree (DFS) | O(n) | O(h) |
| Create commit | Linked list | O(1) | O(1) |
| Get history | Linked list | O(k) | O(k) |
| Find common ancestor | Graph traversal | O(n+m) | O(n) |
| Compute diff | DP (LCS) | O(m×n) | O(m×n) |
| Create branch | Hash table | O(1) | O(1) |
| Switch branch | Hash table | O(1) | O(1) |
| Merge files | Three-way merge | O(n) | O(n) |

Where:
- n = content/file size or number of nodes
- d = directory depth
- h = tree height
- k = number of commits retrieved
- m, n = dimensions for 2D operations

## Conclusion

Mini Git demonstrates efficient use of fundamental DSA concepts:

1. **Hash Tables**: Fast O(1) operations for core functionality
2. **Trees**: Hierarchical data representation
3. **Linked Lists**: Sequential history tracking
4. **Graphs**: Complex relationship modeling
5. **Dynamic Programming**: Optimal substructure for diffs
6. **Graph Traversal**: Exploring connected structures

All algorithms are chosen for their appropriate time/space trade-offs!